# 1. quick start

In [29]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [30]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

将`Dataset`作为参数传递给`DataLoader`。在我们的数据集上包裹了一个可迭代的，并支持自动批处理、采样、洗牌和多进程数据加载。

定义一个64个批次大小，即可迭代数据加载器中的每个元素将返回一批64个功能和标签。

`DataLoader`支持 map 、迭代对数据进行处理。
* map: 是一个 tensor 到 tensor 的映射, 即: X -> y；
* 迭代: 是一个 list(tensor) 的列表, 即: (X, y);

In [31]:
batch_size = 64

training_loader = DataLoader(dataset=training_data, batch_size=batch_size)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size)

for X, y in training_loader: # type: (np.ndarray, np.ndarray)
    print(X.shape, y.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


## mac torch 加速 mps

In [32]:
torch.backends.mps.is_available(), torch.backends.mps.is_built()

(True, True)

In [39]:
device = torch.device('mps')
# 指定 device
torch.rand((3, 2), device=device)

tensor([[0.5061, 0.5847],
        [0.2257, 0.8152],
        [0.3402, 0.5185]], device='mps:0')

# 2. tensor

`tensor`类似于 NumPy 的`ndarray`s，只是张量可以在GPU或其他硬件加速器上运行。
事实上，`tensor`和 NumPy 数组通常可以共享相同的底层内存，无需复制数据。

## tensor 和 numpy 转换

tensor 转换为 numpy 数组

In [34]:
t = torch.ones(5)
t.numpy()

array([1., 1., 1., 1., 1.], dtype=float32)

numpy 数组 转换为 tensor

In [35]:
n = np.ones(5)
torch.from_numpy(n)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)

In [36]:
shape = (2,3)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.9436, 0.6357, 0.1043],
        [0.3591, 0.8148, 0.9719]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [40]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [44]:
if torch.mps:
    tensor = tensor.to('mps')

device(type='mps', index=0)